## World Development Indicators - Analytics

In [161]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

import time

datos = pd.read_csv("suramerica.csv").drop('Unnamed: 0', 1)
paises = ['COL']

preprocessing = 'imput'
search = 'original'
year_init = 2015
year_range = 15
look_back = 3
years = range(year_init-year_range+1,year_init+1)[::-1]

agricultura     = open("Indicadores/iagricultura.txt").read().split(',')
ambiente        = open("Indicadores/iambiente.txt").read().split(',')
ayuda           = open("Indicadores/iayuda.txt").read().split(',')
ciencia         = open("Indicadores/iciencia.txt").read().split(',')
clima           = open("Indicadores/iclima.txt").read().split(',')
comercio        = open("Indicadores/icomercio.txt").read().split(',')
deuda           = open("Indicadores/ideuda.txt").read().split(',')
economia        = open("Indicadores/ieconomia.txt").read().split(',')
educacion       = open("Indicadores/ieducacion.txt").read().split(',')
energia         = open("Indicadores/ienergia.txt").read().split(',')
finanzas        = open("Indicadores/ifinanzas.txt").read().split(',')
genero          = open("Indicadores/igenero.txt").read().split(',')
infraestructura = open("Indicadores/iinfraestructura.txt").read().split(',')
pobreza         = open("Indicadores/ipobreza.txt").read().split(',')
privado         = open("Indicadores/iprivado.txt").read().split(',')
publico         = open("Indicadores/ipublico.txt").read().split(',')
salud           = open("Indicadores/isalud.txt").read().split(',')
social          = open("Indicadores/isocial.txt").read().split(',')
trabajo         = open("Indicadores/itrabajo.txt").read().split(',')
urbano          = open("Indicadores/iurbano.txt").read().split(',')

conjunto_nombre = ['Agricultura','Ambiente','Ayuda','Ciencia','Clima','Comercio','Deuda','Economia','Educacion',
                   'Energia','Finanzas','Genero','Infraestructura','Pobreza','Privado','Publico','Salud','Social',
                   'Trabajo','Urbano']

conjunto = [agricultura,ambiente,ayuda,ciencia,clima,comercio,deuda,economia,educacion,energia,finanzas,genero,
            infraestructura,pobreza,privado,publico,salud,social,trabajo,urbano]

datos.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Antigua and Barbuda,ATG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,126.144000
1,Antigua and Barbuda,ATG,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,88.237117
2,Antigua and Barbuda,ATG,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,7.779958
3,Antigua and Barbuda,ATG,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,80.457159
4,Antigua and Barbuda,ATG,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,32.920000


## Categorizamos los Indicadores

Categorizamos los Indicadores en sus respectivos conjuntos

In [162]:
def categorizar_indicadores(df):
    for i in range(len(conjunto)):
        df.loc[df['IndicatorCode'].isin(conjunto[i]), 'Set'] = conjunto_nombre[i]
    return df

In [163]:
df = categorizar_indicadores(datos)
df.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,Set
0,Antigua and Barbuda,ATG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,126.144000,Social
1,Antigua and Barbuda,ATG,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,88.237117,Salud
2,Antigua and Barbuda,ATG,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,7.779958,Salud
3,Antigua and Barbuda,ATG,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,80.457159,Salud
4,Antigua and Barbuda,ATG,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,32.920000,Salud


## Graficamos Registros Paises

Graficamos la cantidad de años registrados para cada uno de los paises de Suramérica

In [164]:
def registros_country_year(df):
    return df.groupby(['CountryName','Year']).agg(['count']).drop(['CountryCode','IndicatorName','IndicatorCode','Set'], 1)

In [165]:
def graficar_registros_paises(df):

    trace = go.Heatmap(z=df.reset_index(col_level=0)['Value'].values,
                       x=df.reset_index(col_level=0)['Year'],
                       y=df.reset_index(col_level=0)['CountryName'],
                       colorscale = 'Viridis')
    data=[trace]
    
    layout = go.Layout(
        title='Registros Pais + Años',
        xaxis=dict(title='Años',   titlefont=dict(size=18)),
        yaxis=dict(title='Paises', titlefont=dict(size=18))
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig,filename='Registros por Años')

In [147]:
graficar_registros_paises(registros_country_year(df))

## Graficamos Registros Conjuntos

Graficamos los registros de años para cada conjunto de indicadores para Colombia

In [140]:
def registros_set_year(df,pais):
    return df[df['CountryCode'] == pais].groupby(['Set','Year']).agg(['count']).drop(['CountryName','CountryCode','IndicatorName','IndicatorCode'], 1)

In [154]:
def graficar_registros_conjuntos(df,pais):

    trace = go.Heatmap(z=df.reset_index(col_level=0)['Value'].values,
                       x=df.reset_index(col_level=0)['Year'],
                       y=df.reset_index(col_level=0)['Set'],
                       colorscale = 'Viridis')
    data=[trace]
    
    layout = go.Layout(
        title='Registros Conjunto + Años para '+pais,
        xaxis=dict(title='Paises',   titlefont=dict(size=18)),
        yaxis=dict(title='Años', titlefont=dict(size=18))
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig,filename='Registros Conjunto + Años para '+pais)

In [155]:
graficar_registros_conjuntos(registros_set_year(df,'COL'),'Colombia')

## Cargamos Resultados para Colombia y parámetros Originales

Si eliminamos los indicadores con menos de 5 registros y ampliamos el espectro de años a 15 e iniciamos desde 2015 hacia atrás. Tendremos los siguientes resultados:

In [ ]:
resultados = pd.read_csv("colombia_original_log.csv").drop('Unnamed: 0', 1)
resultados

In [ ]:
graficar_resultados(df,1)

In [ ]:
def graficar_resultados(df,look_back):

    trace = go.Heatmap(z=df[df['Look Back'] == look_back]['%'],
                       x=df[df['Look Back'] == look_back]['Base'],
                       y=df[df['Look Back'] == look_back]['Target'],
                       colorscale = 'Viridis')
    data=[trace]
    
    layout = go.Layout(
        title='Total Heatmap'+' with '+'Look Back = '+str(look_back),
        xaxis=dict(title='Base',   titlefont=dict(size=18)),
        yaxis=dict(title='Target', titlefont=dict(size=18))
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig,filename='Total Heatmap'+' with '+'Look Back = '+str(look_back))

## Iteramos todos los Conjuntos de Indicadores

In [ ]:
resultado_global = []

for i in range(len(conjunto)):
    for j in range(len(conjunto)):
        for k in range(1,look_back+1):
            start_time = time.time()
            porcent , results, reg = porcent_result_reg(iterador_global(conjunto[i],conjunto[j],k),conjunto[i],conjunto[j])
            resultado_global.append([conjunto_nombre[i],conjunto_nombre[j],k,reg,porcent,"%s" % (time.time() - start_time),results])
            pd.DataFrame(resultado_global, columns=["Base","Target","Look Back","Reg","%","Time","Results"]).to_csv('colombia_original_log.csv')

pd.DataFrame(resultado_global, columns=["Base","Target","Look Back","Reg","%","Time","Results"])